# Test custom Feature

In [ ]:
import numpy as np
import os
import signalflow as sf
from PIL import Image
from IPython.display import Audio, display
from pixasonics.core import App, Mapper
from pixasonics.features import Feature
from pixasonics.synths import Theremin, Oscillator, FilteredNoise, SimpleFM

In [ ]:
app = App()

In [ ]:
app.load_image_file("images/cellular_dataset/merged_8bit/Timepoint_001_220518-ST_C03_s1.jpg")

In [ ]:
# combine red and green channels and all layers
img_folder = "images/cellular_dataset/single_channel_16bit/"
img_files = os.listdir(img_folder)
imgs_red = [f for f in img_files if f.endswith("w2.TIF")] # only red channel images
imgs_green = [f for f in img_files if f.endswith("w1.TIF")] # only green channel images
imgs = []
for img_red, img_green in zip(imgs_red, imgs_green):
    img_path_red = os.path.join(img_folder, img_red)
    img_path_green = os.path.join(img_folder, img_green)
    img_red = Image.open(img_path_red)
    img_green = Image.open(img_path_green)
    img_red = np.array(img_red)
    img_green = np.array(img_green)
    img = np.stack([img_red, img_green], axis=-1) # now the last dimension is the channel dimension
    imgs.append(img)
img = np.stack(imgs, axis=-1) # now the last dimension is the layer dimension
print(img.shape)
app.load_image_data(img) # load as numpy array

In [ ]:
class MyFeature(Feature):
    def __init__(self, name="MyFeature"):
        super().__init__(name=name)

    def process_image(self, mat):
        return np.random.rand(*mat.shape)
    
    def compute(self, mat):
        num_features = mat.shape[self.target_dim]
        return np.random.rand(num_features)
    
my_feature = MyFeature()
app.attach(my_feature)

In [ ]:
app.detach(my_feature)
my_feature = MyFeature()
my_feature.target_dim = 0
app.attach(my_feature)

In [ ]:
from sklearn.cluster import KMeans

class KMeansFeature(Feature):
    def __init__(self, n_clusters=3, name="KMeansFeature"):
        super().__init__(name=name)
        self.n_clusters = n_clusters
        self.kmeans = None
        self._original_shape = None

    def _reshape_for_kmeans(self, mat):
        """Helper to reshape 4D matrix to 2D for KMeans"""
        mat_reshaped = np.moveaxis(mat, self.target_dim, 0)
        return mat_reshaped.reshape(mat_reshaped.shape[0], -1)

    def process_image(self, mat):
        self._original_shape = mat.shape
        features = self._reshape_for_kmeans(mat)
        self.kmeans = KMeans(n_clusters=self.n_clusters).fit(features.T)

        # Get cluster assignments and reshape back to original dimensions
        labels = self.kmeans.predict(features.T)
        other_dims = [s for i, s in enumerate(mat.shape) if i != self.target_dim]
        self.transformed_image = np.expand_dims(
            labels.reshape(*other_dims), 
            axis=self.target_dim
        )
        return self.transformed_image
    
    def compute(self, mat):
        if self.kmeans is None:
            raise ValueError("KMeans model has not been fitted. Call process_image first.")
        features = self._reshape_for_kmeans(mat)
        labels = self.kmeans.predict(features.T)
        # Compute histogram of cluster assignments
        hist, _ = np.histogram(labels, bins=range(self.n_clusters + 1))
        return hist.astype(float) / hist.sum() # normalize to sum to 1

# Example usage
kmeans_feature = KMeansFeature(n_clusters=10)
app.attach(kmeans_feature)

In [ ]:
# create a multichannel Theremin that has its frequencies in a harmonic series
fundamental_freq = 110
num_harmonics = kmeans_feature.n_clusters
freqs = fundamental_freq * np.arange(1, num_harmonics + 1)
print("Frequencies:",freqs)
osc = Theremin(frequency=freqs, name="KMeansOsc")
app.attach(osc)

# create a Mapper that will map the KMeans cluster histogram to the amplitude of the Theremin 
k2amp = Mapper(kmeans_feature, osc["amplitude"], exponent=1, name="K2Amp")
app.attach(k2amp)

In [ ]:
app.attach(osc)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Create a colormap with distinct colors for each cluster
n_clusters = kmeans_feature.n_clusters
colors = sns.color_palette("husl", n_colors=n_clusters)
colormap = {i: colors[i] for i in range(n_clusters)}

# Get the cluster assignments from transformed_image
cluster_image = kmeans_feature.transformed_image[:, :, 0, 0]
print(cluster_image.shape)

# Create RGB image where each cluster gets a unique color
rgb_image = np.zeros((*cluster_image.shape, 3))
for cluster_id, color in colormap.items():
    mask = cluster_image == cluster_id
    rgb_image[mask] = color

# Plot the results
plt.figure(figsize=(10, 5))

plt.subplot(121)
plt.title('Original Image')
plt.imshow(app.image_displayed)  # Show first layer of original image
plt.axis('off')

plt.subplot(122)
plt.title('K-means Clusters')
plt.imshow(rgb_image)
plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
import json
synthmaps_pca_mel_json = "/Volumes/T7RITMO/synthmaps_code/data/pca_mels_mean.json"
synthmaps_pca_mel_json = "/Volumes/T7RITMO/synthmaps_code/data/pca_perceptual.json"
synthmaps_pca_mel_json = "/Volumes/T7RITMO/synthmaps_code/data/pca_encodec.json"
synthmaps_pca_mel_json = "/Volumes/T7RITMO/synthmaps_code/data/pca_clap.json"
with open(synthmaps_pca_mel_json, "r") as f:
    pca_mel_data = json.load(f)
print(pca_mel_data.keys())

In [ ]:
def fluid_dataset2array(
        dataset: dict,
) -> np.ndarray:
    """
    Convert a json dataset to a numpy array.

    Args:
        dataset (dict): The json dataset to convert.

    Returns:
        np.ndarray: The numpy array.
    """
    num_cols = dataset["cols"]
    num_rows = len(dataset["data"])
    out_array = np.zeros((num_rows, num_cols))
    for i in range(num_rows):
        out_array[i] = np.array(dataset["data"][str(i)])
    return out_array

In [ ]:
pca_mel_data_array = fluid_dataset2array(pca_mel_data)
print(pca_mel_data_array.shape)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

synthmaps_scaler = MinMaxScaler()
pca_mel_data_scaled = synthmaps_scaler.fit_transform(pca_mel_data_array)
print(pca_mel_data_scaled.shape)
print(pca_mel_data_scaled.min(), pca_mel_data_scaled.max())

In [ ]:
fm_params_json = "/Volumes/T7RITMO/synthmaps_code/data/fm_params.json"
with open(fm_params_json, "r") as f:
    fm_params_data = json.load(f)
print(fm_params_data.keys())
fm_params_data_array = fluid_dataset2array(fm_params_data)
print(fm_params_data_array.shape)

In [ ]:
from sklearn.neighbors import KDTree
from sklearn.decomposition import IncrementalPCA

class PCA2FMTimbreSpace(Feature):
    def __init__(self, name="PCA2FMTimbreSpace"):
        super().__init__(name=name)
        self.pca = None
        self.pca_scaler = None
        self.kdtree = KDTree(pca_mel_data_scaled)
        self._original_shape = None
        self._transformed_points = None

    def _reshape_for_pca(self, mat):
        """Helper to reshape 4D matrix (H, W, Ch, L) to 2D by concatenating the Channel and Layer dimensions"""
        mat_reshaped = mat.reshape(mat.shape[0], mat.shape[1], -1)
        return mat_reshaped.reshape(-1, mat_reshaped.shape[-1])

    def process_image(self, mat):
        self._original_shape = mat.shape
        features = self._reshape_for_pca(mat)
        print(features.shape)
        self.pca = IncrementalPCA(n_components=2)
        self.pca.fit(features)
        self.pca_scaler = MinMaxScaler(feature_range=(0.1, 0.9))
        self._transformed_points = self.pca.transform(features)
        self.pca_scaler.fit(self._transformed_points)
        return mat
    
    def compute(self, mat):
        if self.pca is None:
            raise ValueError("PCA model has not been fitted. Call process_image first.")
        features = self._reshape_for_pca(mat)
        projected = self.pca.transform(features)
        projected_scaled = self.pca_scaler.transform(projected)
        projected_scaled_mean = projected_scaled.mean(axis=0, keepdims=True)
        nearest_idx = self.kdtree.query(projected_scaled_mean, return_distance=False)[0][0]
        fm_params = fm_params_data_array[nearest_idx]
        return fm_params


# Example usage
pca2FM_feature = PCA2FMTimbreSpace()
app.attach(pca2FM_feature)

In [ ]:
import matplotlib.pyplot as plt
#plot the pca space
plt.figure(figsize=(10, 5))

plt.subplot(121)

points = pca2FM_feature._transformed_points
plt.scatter(points[:, 0], points[:, 1], alpha=0.5)
plt.title('PCA Space')
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')


In [ ]:
from sklearn.neighbors import KDTree
from sklearn.manifold import TSNE

class TSNE2FMTimbreSpace(Feature):
    def __init__(self, name="TSNE2FMTimbreSpace"):
        super().__init__(name=name)
        self.tsne = None
        self.tsne_scaler = None
        self.kdtree = KDTree(pca_mel_data_scaled)
        self._original_shape = None
        self._transformed_points = None

    def _reshape_for_tsne(self, mat):
        """Helper to reshape 4D matrix (H, W, Ch, L) to 2D by concatenating the Channel and Layer dimensions"""
        mat_reshaped = mat.reshape(mat.shape[0], mat.shape[1], -1)
        return mat_reshaped.reshape(-1, mat_reshaped.shape[-1])

    def process_image(self, mat):
        self._original_shape = mat.shape
        features = self._reshape_for_tsne(mat)
        print(features.shape)
        self.tsne = TSNE(n_components=2)
        self._transformed_points = self.tsne.fit_transform(features)
        self.tsne_scaler = MinMaxScaler(feature_range=(0.1, 0.9))
        self.tsne_scaler.fit(self._transformed_points)
        return mat
    
    def compute(self, mat):
        if self.tsne is None:
            raise ValueError("TSNE model has not been fitted. Call process_image first.")
        features = self._reshape_for_tsne(mat)
        projected = self.tsne.transform(features)
        projected_scaled = self.tsne_scaler.transform(projected)
        projected_scaled_mean = projected_scaled.mean(axis=0, keepdims=True)
        nearest_idx = self.kdtree.query(projected_scaled_mean, return_distance=False)[0][0]
        fm_params = fm_params_data_array[nearest_idx]
        return fm_params


# Example usage
tsne2FM_feature = TSNE2FMTimbreSpace()
app.attach(tsne2FM_feature)

In [ ]:
import matplotlib.pyplot as plt
#plot the tsne space
plt.figure(figsize=(10, 5))

plt.subplot(121)

points = tsne2FM_feature._transformed_points
plt.scatter(points[:, 0], points[:, 1], alpha=0.5)
plt.title('TSNE Space')
plt.xlabel('TSNE 1')
plt.ylabel('TSNE 2')

In [ ]:
fm = SimpleFM()
app.attach(fm)

In [ ]:
class FMParamSetter(Mapper):
    def __init__(self, feature, synth, name="FMParamSetter"):
        super().__init__(feature, synth, name=name)

    def map(self, frame=None):
        fm_params = self.buf_in.data
        if fm_params.shape[0] == 3:
            self.obj_out_owner.set_input_buf("carrier_freq", fm_params[0], from_slider=False)
            self.obj_out_owner.set_input_buf("harm_ratio", fm_params[1], from_slider=False)
            self.obj_out_owner.set_input_buf("mod_index", fm_params[2], from_slider=False)

fm_param_setter = FMParamSetter(pca2FM_feature, fm["carrier_freq"])
app.attach(fm_param_setter)

In [ ]:
fm2 = SimpleFM(name="FM2")
app.attach(fm2)
fm2_param_setter = FMParamSetter(tsne2FM_feature, fm2["carrier_freq"])
app.attach(fm2_param_setter)

In [ ]:
app.probe_height, app.probe_width = 20, 20

In [ ]:
app.detach(pca2FM_feature)
app.detach(fm_param_setter)

In [ ]:
app.detach(pca2FM_feature)
app.detach(fm_param_setter)
pca2FM_feature = PCA2FMTimbreSpace()
pca2FM_feature.filter_channels = None # try also 0
app.attach(pca2FM_feature)
fm_param_setter = FMParamSetter(pca2FM_feature, fm["carrier_freq"])
app.attach(fm_param_setter)

# Test custom Synth

In [1]:
import signalflow as sf
import numpy as np
import json
from typing import List, Dict
from pixasonics.utils import broadcast_params, ParamSliderDebouncer, array2str
from pixasonics.synths import Mixer
from pixasonics.ui import SynthCard, find_widget_by_tag
import copy

In [2]:
def patch_spec2dict(spec: sf.PatchSpec) -> Dict:
    """
    Convert a patch spec to a python dict.
    """
    spec_dict = json.loads(spec.to_json())
    return spec_dict

def patch_dict2spec(spec_dict: Dict) -> sf.PatchSpec:
    """
    Convert a python dict to a patch spec.
    """
    spec = sf.PatchSpec.from_json(json.dumps(spec_dict))
    return spec

def get_spec_input_names(spec: sf.PatchSpec) -> List[str]:
    """
    Get the inputs of a patch spec.
    """
    spec_dict = patch_spec2dict(spec)
    inputs = spec_dict["inputs"]
    input_names = [input["patch_input_name"] for input in inputs]
    return input_names

def find_dict_with_entry(list_of_dicts: List[Dict], key: str, value) -> Dict:
    """
    Find a dictionary in a list where a specific key has a given value.
    
    Args:
        list_of_dicts (list): A list of dictionaries to search through
        key (str): The key to search for in each dictionary
        value: The value to match against the key
        
    Returns:
        dict: The first dictionary where dict[key] == value, or None if no match is found
    """
    return next((d for d in list_of_dicts if key in d and d[key] == value), None)

def get_spec_inputs_dict(spec: sf.PatchSpec) -> Dict:
    """
    Get the inputs of a patch spec as a dict, where keys are the
    'patch_input_name' and values are the corresponding node values.
    """
    spec_dict = patch_spec2dict(spec)
    inputs = spec_dict["inputs"]
    nodes = spec_dict["nodes"]
    out_dict = {}
    for input in inputs:
        input_name = input["patch_input_name"]
        node_id = input["node_id"]
        node_input_name = input["node_input_name"]
        # find the node with the same id
        node = find_dict_with_entry(nodes, "id", node_id)
        input_value = node["inputs"][node_input_name]
        # if the value is a dict it means there is a channel-array node that holds the values
        if isinstance(input_value, dict):
            channel_array_id = input_value["id"]
            # find the channel array node
            channel_array_node = find_dict_with_entry(nodes, "id", channel_array_id)
            # check that it is really a channel array node
            assert channel_array_node["node"] == "channel-array"
            # get the values
            input_value = channel_array_node["inputs"] # this is a dict
            # convert to list
            input_value = [val for val in input_value.values()]
        out_dict[input_name] = input_value
    return out_dict

def get_patch_spec_num_output_channels(spec: sf.PatchSpec) -> int:
    """
    Get the number of output channels of a patch spec.
    """
    patch = sf.Patch(spec)
    return patch.output.num_output_channels

def pretty_print_dict(d):
    """Format dictionary as pretty JSON with indentation and line breaks"""
    print(json.dumps(d, indent=4))

In [3]:
# create a graph
graph = sf.AudioGraph(start=True)

[miniaudio] Output device: MacBook Pro Speakers (48000Hz, buffer size 480 samples, 2 channels)


In [4]:
# create a test patch
class TestPatch(sf.Patch):
    def __init__(self, freq=440):
        super().__init__()
        freq = self.add_input("freq", freq)
        out = sf.SineOscillator(freq)
        self.set_output(out)

In [5]:
my_patch = TestPatch()

In [6]:
my_patch_multi = TestPatch(freq=[440, 441, 442])

In [7]:
my_spec_multi = my_patch_multi.to_spec()

In [8]:
my_spec = my_patch.to_spec()

In [9]:
get_spec_inputs_dict(my_spec)

{'freq': 440}

In [10]:
get_spec_inputs_dict(my_spec_multi)

{'freq': [440, 441, 442]}

In [11]:
get_spec_input_names(my_spec)

['freq']

In [12]:
get_patch_spec_num_output_channels(my_spec)

1

In [13]:
PARAM_SLIDER_DEBOUNCE_TIME = 0.05

In [ ]:
class Synth():
    def __init__(
            self, 
            patch_spec: sf.PatchSpec, 
            params_dict: Dict = None, 
            name: str = "Synth",
            add_amplitude: bool = True,
            add_panning: bool = True):
        self.name = name
        self.patch_spec = patch_spec
        self.params = {} if params_dict is None else copy.deepcopy(params_dict)
        self.synth = None
        self.num_channels = -1 # will be set in generate_params
        self.add_amplitude = add_amplitude
        self.add_panning = add_panning
        self.id = str(id(self))
        # generate params dict
        self.generate_params()
        # create param buffers, their players, smoothing, and the patch
        self.create_audio_graph()
        # create ui
        self.create_ui()
        # create param slider debouncer if necessary
        self.debouncer = ParamSliderDebouncer(PARAM_SLIDER_DEBOUNCE_TIME) if self.num_channels == 1 else None


    def generate_params(self):
        # check that there are keys for all params
        params = get_spec_inputs_dict(self.patch_spec)
        # add amplitude if requested
        if self.add_amplitude:
            # amplitude should default to 0.5
            params["amplitude"] = 0.5
            self.params["amplitude"] = {
                "min": 0,
                "max": 1,
            }
        # add panning if requested
        if self.add_panning:
            # panning should default to middle if single channel or a spread if multichannel
            spec_output_channels = get_patch_spec_num_output_channels(self.patch_spec)
            if spec_output_channels == 1:
                panning_default = 0
            else:
                panning_default = [-1, 1]
            params["panning"] = panning_default
            self.params["panning"] = {
                "min": -1,
                "max": 1,
            }

        for param_name in params.keys():
            # if not, then make a new dict for it
            if param_name not in self.params:
                self.params[param_name] = {}
            # if yes, then assert that it is a dict
            assert isinstance(self.params[param_name], dict)
            # fill in the dict with the values
            template = {
                "min": 0,
                "max": 1,
                "unit": "",
                "scale": "linear",
            }
            forced_template = {
                "buffer": None,
                "buffer_player": None,
                "name": f"{self.name} {param_name.capitalize()}",
                "param_name": param_name,
                "owner": self,
            }
            # combine template with user-provided values
            self.params[param_name] = {**template, **self.params[param_name]} # user can overwrite the template
            # combine forced_template with user-provided values
            self.params[param_name] = {**self.params[param_name], **forced_template} # user cannot overwrite the forced_template

        # broadcast params
        param_names = list(params.keys())
        param_values = [params[param_name] for param_name in param_names]
        broadcasted_params = broadcast_params(*param_values)
        self.num_channels = len(broadcasted_params[0]) # all params should have the same amount of channels now
        # register values in params dict
        for i, param_name in enumerate(param_names):
            params_list = broadcasted_params[i]
            self.params[param_name]["default"] = params_list
            self.params[param_name]["value"] = params_list
    

    def create_audio_graph(self):
        graph = sf.AudioGraph.get_shared_graph()
        mix_val = sf.calculate_decay_coefficient(0.05, graph.sample_rate, 0.001)
        self.patch = sf.Patch(self.patch_spec)

        param_names = list(self.params.keys())
        for param_name in param_names:
            params_list = self.params[param_name]["value"] # expecting the broadcasted values
            buffer = sf.Buffer(self.num_channels, 1)
            buffer.data[:, :] = np.array(params_list).reshape(self.num_channels, 1)
            self.params[param_name]["buffer"] = buffer
            buffer_player = sf.BufferPlayer(buffer, loop=True)
            self.params[param_name]["buffer_player"] = buffer_player
            smoothed = sf.Smooth(buffer_player, mix_val)
            self.params[param_name]["smoothed"] = smoothed
            # set the input of the patch to the smoothed value, except the added amplitude and panning params
            if self.add_amplitude and param_name == "amplitude":
                continue
            if self.add_panning and param_name == "panning":
                continue
            self.patch.set_input(param_name, smoothed)

        self.patch_output = self.patch.output * self.params["amplitude"]["smoothed"] if self.add_amplitude else self.patch.output
        self.output = Mixer(self.patch_output, self.params["panning"]["smoothed"] * 0.5 + 0.5, out_channels=2) if self.add_panning else self.patch_output
        

    def set_input_buf(self, name, value, from_slider=False):
        self.params[name]["buffer"].data[:, :] = value
        self.params[name]["value"] = value.tolist() if isinstance(value, np.ndarray) else value
        if not from_slider and self.num_channels == 1:
            slider = find_widget_by_tag(self.ui, name)
            slider.unobserve_all()
            slider_value = value if self.num_channels == 1 else array2str(value)
            self.debouncer.submit(name, lambda: self.update_slider(slider, slider_value))
        elif not from_slider and self.num_channels > 1:
            slider = find_widget_by_tag(self.ui, name)
            slider.value = array2str(value)


    def update_slider(self, slider, value):
        slider.unobserve_all()
        slider.value = value
        slider.observe(
            lambda change: self.set_input_buf(
                    change["owner"].tag, 
                    change["new"],
                    from_slider=True
                ), 
                names="value")
        

    def reset_to_default(self):
        for param in self.params:
            self.set_input_buf(param, np.array(self.params[param]["default"]).reshape(self.num_channels, 1), from_slider=False)


    def __getitem__(self, key):
        return self.params[key]
    

    def create_ui(self):
        self._ui = SynthCard(
            name=self.name,
            id=self.id,
            params=self.params,
            num_channels=self.num_channels
        )
        self._ui.synth = self


    @property
    def ui(self):
        return self._ui()
    

    def __repr__(self):
        return f"Synth {self.id}: {self.name}"



In [15]:
params = {
    "freq": {
        "min": 20,
        "max": 20000,
        "unit": "Hz",
        "scale": "log",
    },
}

In [16]:
params

{'freq': {'min': 20, 'max': 20000, 'unit': 'Hz', 'scale': 'log'}}

In [17]:
graph.clear()

In [18]:
main_bus = sf.Bus(num_channels=2)
main_bus.play()

In [19]:
graph.status

'AudioGraph: 1 active node, 0 patches, 0.12% CPU usage, 0.1MB memory usage, output = -dB'

In [20]:
pixa_synth = Synth(my_spec, params_dict=params, name="Laszlo")

In [21]:
main_bus.add_input(pixa_synth.output)

In [23]:
main_bus.remove_input(pixa_synth.output)

In [22]:
pixa_synth.ui

Box(children=(Box(children=(Label(value='Laszlo', style=LabelStyle(font_size='20px', font_weight='bold')), Lab…

In [24]:
pixa_synth.params["freq"]["value"]

1602.9869828846236

In [25]:
pixa_synth.params["freq"]["smoothed"].get_value()

1602.947998046875

In [ ]:
# test redoing the graph with a different sample rate/buffer size and re-creating the synth's audio graph
graph.destroy()
config = sf.AudioGraphConfig()
config.sample_rate = 48000
config.output_buffer_size = 256
graph = sf.AudioGraph(config=config, start=True)
main_bus = sf.Bus(num_channels=2)
main_bus.play()
# pixa_synth.create_audio_graph()
# main_bus.add_input(pixa_synth.output)
graph.sample_rate, graph.output_buffer_size

In [26]:
pixa_synth_multi = Synth(my_spec_multi, params_dict=params)

In [27]:
pixa_synth_multi.ui

Box(children=(Box(children=(Label(value='PixaSynth', style=LabelStyle(font_size='20px', font_weight='bold')), …

In [28]:
main_bus.add_input(pixa_synth_multi.output)

In [30]:
main_bus.remove_input(pixa_synth_multi.output)

In [29]:
new_freqs = np.array([440, 460, 480]).reshape(3, 1)
pixa_synth_multi.set_input_buf("freq", new_freqs)

In [31]:
class SimpleFMPatch(sf.Patch):
    def __init__(
            self, 
            carrier_frequency=440, 
            harmonicity_ratio=1,
            modulation_index=1,
            lp_cutoff=20000,
            lp_resonance=0.5,
            hp_cutoff=20,
            hp_resonance=0.5,
            ):
        super().__init__()
        carrier_freq = self.add_input("carrier_freq", carrier_frequency)
        harm_ratio = self.add_input("harm_ratio", harmonicity_ratio)
        mod_index = self.add_input("mod_index", modulation_index)
        lp_cutoff = self.add_input("lp_cutoff", lp_cutoff)
        lp_resonance = self.add_input("lp_resonance", lp_resonance)
        hp_cutoff = self.add_input("hp_cutoff", hp_cutoff)
        hp_resonance = self.add_input("hp_resonance", hp_resonance)
        # create the synth
        mod_freq = carrier_freq * harm_ratio
        mod_amp = mod_freq * mod_index
        modulator = sf.SineOscillator(mod_freq) * mod_amp
        carrier = sf.SineOscillator(carrier_freq + modulator)
        # create the filters
        lp_resonance_clipped = sf.Clip(lp_resonance, 0.0, 0.999)
        hp_resonance_clipped = sf.Clip(hp_resonance, 0.0, 0.999)
        lp = sf.SVFilter(
            carrier,
            filter_type="low_pass",
            cutoff=lp_cutoff,
            resonance=lp_resonance_clipped
        )
        hp = sf.SVFilter(
            lp,
            filter_type="high_pass",
            cutoff=hp_cutoff,
            resonance=hp_resonance_clipped
        )
        out = hp
        self.set_output(out)

In [32]:
carr_freqs = [440, 480, 520]
simple_fm_patch = SimpleFMPatch(carrier_frequency=carr_freqs)
simple_fm_patch_spec = simple_fm_patch.to_spec()

In [33]:
fm_params = {
    "carrier_freq": {
        "min": 20,
        "max": 20000,
        "unit": "Hz",
        "scale": "log",
    },
    "harm_ratio": {
        "min": 0,
        "max": 10,
    },
    "mod_index": {
        "min": 0,
        "max": 10,
    },
    "lp_cutoff": {
        "min": 20,
        "max": 20000,
        "unit": "Hz",
        "scale": "log",
    },
    "lp_resonance": {
        "min": 0,
        "max": 0.999,
    },
    "hp_cutoff": {
        "min": 20,
        "max": 20000,
        "unit": "Hz",
        "scale": "log",
    },
    "hp_resonance": {
        "min": 0,
        "max": 0.999,
    },
}

In [34]:
pixa_fm = Synth(simple_fm_patch_spec, params_dict=fm_params, name="SimpleFM")

In [35]:
pixa_fm.ui

Box(children=(Box(children=(Label(value='SimpleFM', style=LabelStyle(font_size='20px', font_weight='bold')), L…

In [41]:
main_bus.add_input(pixa_fm.output)

In [42]:
main_bus.remove_input(pixa_fm.output)

In [43]:
graph.status

'AudioGraph: 1 active node, 0 patches, 0.11% CPU usage, 3.9MB memory usage, output = -dB'

In [38]:
pixa_fm.set_input_buf("carrier_freq", np.array([440, 550, 660]).reshape(3, 1))
pixa_fm.set_input_buf("harm_ratio", np.array([1, 2, 3]).reshape(3, 1))
pixa_fm.set_input_buf("mod_index", np.array([1, 2, 3]).reshape(3, 1))

In [40]:
pixa_fm.create_audio_graph()

In [44]:
class SimpleFM(Synth):
    def __init__(
        self,
        carrier_frequency=440, 
        harmonicity_ratio=1,
        modulation_index=1,
        lp_cutoff=20000,
        lp_resonance=0.5,
        hp_cutoff=20,
        hp_resonance=0.5,
        name="SimpleFM",
    ):
        # create the patch spec
        _spec = SimpleFMPatch(
            carrier_frequency=carrier_frequency, 
            harmonicity_ratio=harmonicity_ratio,
            modulation_index=modulation_index,
            lp_cutoff=lp_cutoff,
            lp_resonance=lp_resonance,
            hp_cutoff=hp_cutoff,
            hp_resonance=hp_resonance
        ).to_spec()
        # create the params dict
        _params = {
            "carrier_freq": {
                "min": 20,
                "max": 20000,
                "unit": "Hz",
                "scale": "log",
            },
            "harm_ratio": {
                "min": 0,
                "max": 10,
            },
            "mod_index": {
                "min": 0,
                "max": 10,
            },
            "lp_cutoff": {
                "min": 20,
                "max": 20000,
                "unit": "Hz",
                "scale": "log",
            },
            "lp_resonance": {
                "min": 0,
                "max": 0.999,
            },
            "hp_cutoff": {
                "min": 20,
                "max": 20000,
                "unit": "Hz",
                "scale": "log",
            },
            "hp_resonance": {
                "min": 0,
                "max": 0.999,
            },
        }
        # call the parent constructor
        super().__init__(_spec, params_dict=_params, name=name)
    
    def __repr__(self):
        return f"SimpleFM {self.id}: {self.name}"

In [45]:
test_fm = SimpleFM(carrier_frequency=[440, 550, 660])

In [46]:
test_fm.ui

Box(children=(Box(children=(Label(value='SimpleFM', style=LabelStyle(font_size='20px', font_weight='bold')), L…

In [47]:
graph.status

'AudioGraph: 1 active node, 0 patches, 0.11% CPU usage, 5.8MB memory usage, output = -dB'

In [48]:
main_bus.add_input(test_fm.output)

In [49]:
main_bus.remove_input(test_fm.output)

In [50]:
class OscillatorPatch(sf.Patch):
    def __init__(
            self, 
            frequency=440, 
            lp_cutoff=20000,
            lp_resonance=0.5,
            hp_cutoff=20,
            hp_resonance=0.5,
            waveform="sine",
            ):
        super().__init__()
        wf_types = ["sine", "square", "saw", "triangle"]
        assert waveform in wf_types, f"Waveform must be one of {wf_types}"
        frequency = self.add_input("frequency", frequency)
        lp_cutoff = self.add_input("lp_cutoff", lp_cutoff)
        lp_resonance = self.add_input("lp_resonance", lp_resonance)
        hp_cutoff = self.add_input("hp_cutoff", hp_cutoff)
        hp_resonance = self.add_input("hp_resonance", hp_resonance)
        # create the synth
        osc_templates = [sf.SineOscillator, sf.SquareOscillator, sf.SawOscillator, sf.TriangleOscillator]
        osc = osc_templates[wf_types.index(waveform)](frequency)
        # create the filters
        lp_resonance_clipped = sf.Clip(lp_resonance, 0.0, 0.999)
        hp_resonance_clipped = sf.Clip(hp_resonance, 0.0, 0.999)
        lp = sf.SVFilter(
            osc,
            filter_type="low_pass",
            cutoff=lp_cutoff,
            resonance=lp_resonance_clipped
        )
        hp = sf.SVFilter(
            lp,
            filter_type="high_pass",
            cutoff=hp_cutoff,
            resonance=hp_resonance_clipped
        )
        out = hp
        self.set_output(out)

In [51]:
class Oscillator(Synth):
    def __init__(
        self,
        frequency=440, 
        lp_cutoff=20000,
        lp_resonance=0.5,
        hp_cutoff=20,
        hp_resonance=0.5,
        waveform="sine",
        name="Oscillator",
        ):
        # create the patch spec
        _spec = OscillatorPatch(
            frequency=frequency, 
            lp_cutoff=lp_cutoff,
            lp_resonance=lp_resonance,
            hp_cutoff=hp_cutoff,
            hp_resonance=hp_resonance,
            waveform=waveform
        ).to_spec()
        # create the params dict
        _params = {
            "frequency": {
                "min": 20,
                "max": 20000,
                "unit": "Hz",
                "scale": "log",
            },
            "lp_cutoff": {
                "min": 20,
                "max": 20000,
                "unit": "Hz",
                "scale": "log",
            },
            "lp_resonance": {
                "min": 0,
                "max": 0.999,
            },
            "hp_cutoff": {
                "min": 20,
                "max": 20000,
                "unit": "Hz",
                "scale": "log",
            },
            "hp_resonance": {
                "min": 0,
                "max": 0.999,
            },
        }
        # call the parent constructor
        super().__init__(_spec, params_dict=_params, name=name)

    def __repr__(self):
        return f"Oscillator {self.id}: {self.name}"

In [52]:
test_osc = Oscillator(waveform="saw", frequency=[440, 550, 660])

In [53]:
test_osc.ui

Box(children=(Box(children=(Label(value='Oscillator', style=LabelStyle(font_size='20px', font_weight='bold')),…

In [54]:
main_bus.add_input(test_osc.output)

In [56]:
test_osc.set_input_buf("frequency", np.array([440, 550, 660]).reshape(3, 1) / 2)

In [58]:
main_bus.remove_input(test_osc.output)

In [57]:
graph.status

'AudioGraph: 36 active nodes, 0 patches, 5.61% CPU usage, 7.3MB memory usage, output = -9.3dB'

In [59]:
class ThereminPatch(sf.Patch):
    def __init__(self, frequency=440):
        super().__init__()
        frequency = self.add_input("frequency", frequency)
        out = sf.SineOscillator(frequency)
        self.set_output(out)

In [60]:
class Theremin(Synth):
    def __init__(self, frequency=440, name="Theremin"):
        # create the patch spec
        _spec = ThereminPatch(
            frequency=frequency, 
        ).to_spec()
        # create the params dict
        _params = {
            "frequency": {
                "min": 20,
                "max": 20000,
                "unit": "Hz",
                "scale": "log",
            },
        }
        # call the parent constructor
        super().__init__(_spec, params_dict=_params, name=name)
        
    def __repr__(self):
        return f"Theremin {self.id}: {self.name}"

In [61]:
test_theremin = Theremin(frequency=[440, 550, 660])
test_theremin.ui

Box(children=(Box(children=(Label(value='Theremin', style=LabelStyle(font_size='20px', font_weight='bold')), L…

In [62]:
main_bus.add_input(test_theremin.output)

In [65]:
test_theremin.set_input_buf("frequency", np.array([440, 550, 660]).reshape(3, 1) / 4)

In [66]:
main_bus.remove_input(test_theremin.output)

In [67]:
class FilteredNoisePatch(sf.Patch):
    def __init__(
            self,
            filter_type="band_pass", # can be 'low_pass', 'band_pass', 'high_pass', 'notch', 'peak', 'low_shelf', 'high_shelf'
            order=3,
            cutoff=440,
            resonance=0.5,
            ):
        super().__init__()
        filter_types = ["low_pass", "band_pass", "high_pass", "notch", "peak", "low_shelf", "high_shelf"]
        assert filter_type in filter_types, f"Filter type must be one of {filter_types}"
        self.filter_type = filter_type
        self.order = np.clip(order, 1, 8)
        cutoff = self.add_input("cutoff", cutoff)
        resonance = self.add_input("resonance", resonance)
        resonance_clipped = sf.Clip(resonance, 0.0, 0.999)
        graph = sf.AudioGraph.get_shared_graph()
        mix_val = sf.calculate_decay_coefficient(0.05, graph.sample_rate, 0.001)
        # create the synth
        noise = sf.WhiteNoise()
        # first one
        filters = sf.SVFilter(
            noise,
            filter_type=self.filter_type,
            cutoff=cutoff,
            resonance=resonance_clipped
        )
        # the rest
        for i in range(1, self.order):
            filters = sf.SVFilter(
                filters,
                filter_type=self.filter_type,
                cutoff=cutoff,
                resonance=resonance_clipped
            )
        # amplitude compensation
        filters_rms = sf.RMS(filters)
        filters_rms_smooth = sf.Smooth(filters_rms, mix_val)
        filters = filters / filters_rms_smooth
        # output
        out = filters
        self.set_output(out * 0.707 * 0.5)

In [68]:
class FilteredNoise(Synth):
    def __init__(
            self,
            filter_type="band_pass", # can be 'low_pass', 'band_pass', 'high_pass', 'notch', 'peak', 'low_shelf', 'high_shelf'
            order=3,
            cutoff=440,
            resonance=0.5,
            name="FilteredNoise",
            ):
        # create the patch spec
        _spec = FilteredNoisePatch(
            filter_type=filter_type,
            order=order,
            cutoff=cutoff,
            resonance=resonance,
        ).to_spec()
        # create the params dict
        _params = {
            "cutoff": {
                "min": 20,
                "max": 20000,
                "unit": "Hz",
                "scale": "log",
            },
            "resonance": {
                "min": 0,
                "max": 0.999,
            },
        }
        # call the parent constructor
        super().__init__(_spec, params_dict=_params, name=name)

    def __repr__(self):
        return f"FilteredNoise {self.id}: {self.name}"

In [69]:
test_fnoise = FilteredNoise(cutoff=[440, 550, 660], resonance=0.99, order=8)
test_fnoise.ui

Box(children=(Box(children=(Label(value='FilteredNoise', style=LabelStyle(font_size='20px', font_weight='bold'…

In [70]:
main_bus.add_input(test_fnoise.output)

In [71]:
test_fnoise.set_input_buf("cutoff", np.array([440, 550, 660]).reshape(3, 1) / 2)

In [72]:
main_bus.remove_input(test_fnoise.output)

In [73]:
graph.status

'AudioGraph: 1 active node, 0 patches, 0.15% CPU usage, 11.0MB memory usage, output = -dB'

# Test hanging bug

In [ ]:
import signalflow as sf

graph = sf.AudioGraph.get_shared_graph()
if graph is not None:
    graph.destroy()
    graph = None
if graph is None:
    graph = sf.AudioGraph(start=True)

node_1 = sf.SineOscillator(440)
node_2 = sf.Constant(0.1)
node_3 = node_1 * node_2

In [ ]:
node_3.play()

In [ ]:
node_3.stop()

In [ ]:
node_1.play()

In [ ]:
node_1.stop()

In [ ]:
import signalflow as sf

graph = sf.AudioGraph.get_shared_graph()
if graph is not None:
    graph.destroy()
    graph = None
if graph is None:
    graph = sf.AudioGraph(start=True)

my_bus = sf.Bus(1)
node_1 = sf.SineOscillator(440)
node_2 = sf.Constant(0.1)
node_3 = node_1 * node_2
node_4 = sf.SineOscillator(550)

my_bus.play()

In [ ]:
graph.status

In [ ]:
graph.clear()

In [ ]:
graph.stop()

In [ ]:
graph.start()

In [ ]:
my_bus.add_input(node_3)

In [ ]:
my_bus.remove_input(node_3)

In [ ]:
my_bus.add_input(node_4)

In [ ]:
my_bus.remove_input(node_4)

In [ ]:
my_bus.add_input(node_3)
my_bus.remove_input(node_3)
my_bus.add_input(node_4)
my_bus.remove_input(node_4)

In [ ]:
my_bus.add_input(node_1)

In [ ]:
my_bus.remove_input(node_1)